# 回帰結合型のニューラルネットワークによる文章生成

---
## 目的
回帰結合型のニューラルネットワーク，すなわち再帰型ニューラルネットワーク (Recurrent Neural Network; RNN) を用いてPenn Tree Bankデータセットに対する次単語の予測を行う．
また，教師強制の有無による性能の違いを確認する．


## 対応するチャプター
* 10.2: 教師強制と出力回帰のあるネットワーク
* 10.2: 回帰結合型ネットワークにおける勾配計算（BPTT）
* 10.10.1: LSTM
* 10.10.2: GRU
* 10.11: 勾配のクリッピング


## モジュールのインポート
プログラムの実行に必要なモジュールをインポートします．

In [ ]:
from time import time
import numpy as np

import chainer
from chainer.datasets import get_ptb_words, get_ptb_words_vocabulary
from chainer import cuda
from chainer import Variable
import chainer.functions as F
import chainer.links as L
from chainer.optimizer_hooks import GradientClipping

## GPUの確認
GPUを使用した計算が可能かどうかを確認します．

`GPU avilability: True`と表示されれば，GPUを使用した計算をChainerで行うことが可能です．
Falseとなっている場合は，上記の「Google Colaboratoryの設定確認・変更」に記載している手順にしたがって，設定を変更した後に，モジュールのインポートから始めてください．

In [ ]:
print('GPU availability:', chainer.cuda.available)
print('cuDNN availablility:', chainer.cuda.cudnn_enabled)

## データセットの読み込み

Penn Tree Bank (PTB) データセットを読み込みます．

読み込んだ学習データのサイズを確認します．
学習，検証，テストデータはそれぞれ929589，73760，82430のサイズの1次元配列になっていることがわかります．

また，`get_ptb_words_vocabulary`関数を用いて，ptbデータセットに存在する英単語の情報を取得します．
`vocab`には英単語とその単語を示すIDが辞書型のオブジェクトとして格納されています．
英単語の数は10000です．

最後に，keyと値の組み合わせを逆にした辞書`inverse_vocab`を作成します．
これはIDで出力された予測結果から英単語を検索する際に使用します．

In [ ]:
# データセットの読み込み
train, val, test = get_ptb_words()
print(train.shape, val.shape, test.shape)

# 単語（vocabulary）の確認
vocab = get_ptb_words_vocabulary()
print(len(vocab))

# 逆引きの辞書を作成
inverse_vocab = {v:k for k, v in vocab.items()}

### Benn Tree Bankデータセットの表示

PTBデータセットの中身を`print`関数を使って表示してみます．

学習用データを表示すると，1次元配列に整数値が格納されていることがわかります．

また，`vocab`のうち，英単語を指定すると，各英単語に対応するIDガ表示されます．

In [ ]:
print("train sentence:", train)
print(vocab['player'], vocab['primarily'], vocab['arose'], vocab['generate'], vocab['partnership'])

## ネットワークモデルの定義
再帰型ニューラルネットワークを定義します．

ここでは，埋め込み層1層，LSTM層1層，全結合層1層から構成されるネットワークとします．

`reset_state`関数では，LSTM層が持つ，内部状態（隠れ状態・セル状態）を初期化します．

次に，`__call__`関数では，定義した層を接続して処理するように記述します．
`__call__`関数の引数`x`は入力データ（単語のID）です．
入力データは`embed`にて，入力された単語のIDから入力された単語を表現するベクトルを生成します．
その後，LSTM，全結合層へと入力することで，入力された単語の次の単語を予測結果として出力します．
その際，LSTMおよび全結合層からの出力にはdropoutを適用しており，過学習の抑制を図っています．

In [ ]:
class RNNLM(chainer.Chain):

    def __init__(self, n_vocab, n_units):
        super(RNNLM, self).__init__()
        with self.init_scope():
            self.embed = L.EmbedID(n_vocab, n_units)
            self.l1 = L.LSTM(n_units, n_units)
            self.l2 = L.Linear(n_units, n_vocab)

        for param in self.params():
            param.array[...] = np.random.uniform(-0.1, 0.1, param.shape)

    def reset_state(self):
        self.l1.reset_state()

    def __call__(self, x):
        h0 = self.embed(x)
        h1 = self.l1(F.dropout(h0))
        y = self.l2(F.dropout(h1))
        return y

## ネットワークの作成
上のプログラムで定義したネットワークを作成します．
ここでは，GPUで学習を行うために，modelをGPUに送るto_gpu関数を利用しています．

学習を行う際の最適化方法としてモーメンタムSGD(モーメンタム付き確率的勾配降下法）を利用します．また，学習率を1.0として引数に与えます．そして，最適化方法のsetup関数にネットワークモデルを与えます．

また，勾配の爆発により学習の不安定性に対応するため，勾配のクリッピングを行います．
最適化手法を設定した`optimizer_1`に`add_hook`メソッドを用いて学習を行う際の条件を追加します．
ここでは，勾配のクリッピングを行う`GradientClipping`関数を追加します．

In [ ]:
num_vocab = len(vocab)
num_units = 1024
model_1 = RNNLM(n_vocab=num_vocab, n_units=num_units)
model_1.to_gpu()

optimizer_1 = chainer.optimizers.MomentumSGD(lr=1.0, momentum=0.9)
optimizer_1.setup(model_1)
optimizer_1.add_hook(GradientClipping(5.0))

次に，GPUに対応した行列演算モジュールのcupyを呼び出し，学習およびテストデータをcupyの形式に変換します．
cupyはnumpyと互換性があります．

先ほど読み込んだPTBデータセットを学習に使用するために，データを整理します．
まず`bproplen`でネットワークへ入力するデータの長さを指定します．
その後，学習データを指定した長さに区切ることで，学習サンプルを作成します．

In [ ]:
xp = cuda.cupy

bproplen = 35

train_x, train_y = [], []
for idx_window in range(0, len(train) - bproplen - 1, 10):
    train_x.append(train[idx_window:idx_window + bproplen])
    train_y.append(train[idx_window + 1:idx_window + bproplen + 1])
train_x = xp.array(train_x, dtype=xp.int32)
train_y = xp.array(train_y, dtype=xp.int32)

val = xp.array(val, dtype=xp.int32)
test = xp.array(test, dtype=xp.int32)

## 学習（教師強制; Teacher forcing）

教師強制の方法でネットワークを学習します．
教師強制ではネットワークへの入力データとして，教師データ（正しい英単語）を順番に入力し，出力と教師ラベルとの誤差を用いて学習する方法です．

１回の誤差を算出するデータ数（ミニバッチサイズ）128，学習エポック数を100とします．
先ほど作成したの学習データサイズを取得し，1エポック内における更新回数を求めます．
学習データは毎エポックでランダムに利用するため，numpyの`permutation`という関数を利用します．
各更新において，学習用データと教師データをそれぞれ`x`と`t`とし，`to_gpu`関数でGPUに転送します．
学習モデルにxを与えて各クラスの確率`y`を取得します．
各クラスの確率`y`と教師ラベル`t`との誤差を`softmax_coross_entropy`誤差関数で算出します．
そして，誤差を`backward`関数で逆伝播し，ネットワークの更新を行います．

In [ ]:
# ミニバッチサイズ・エポック数．学習データ数の設定
batch_size = 128
epoch_num = 100
train_data_num = train_x.shape[0]
num_iter_per_epoch = int(train_data_num / batch_size)


# 学習の実行
start = time()
for epoch in range(1, epoch_num + 1):
    
    sum_loss = 0
    
    perm = xp.random.permutation(train_data_num)
    
    for i in range(0, train_data_num, batch_size):
        
        x = Variable(cuda.to_gpu(train_x[perm[i:i+batch_size]]))
        t = Variable(cuda.to_gpu(train_y[perm[i:i+batch_size]]))
        
        accum_loss = 0
        model_1.reset_state()
        
        for idx_window in range(bproplen):
            
            y = model_1(x[:, idx_window])
            
            loss = F.softmax_cross_entropy(y, t[:, idx_window])
            accum_loss += loss
            sum_loss += loss.data

        optimizer_1.target.cleargrads()
        accum_loss.backward()
        accum_loss.unchain_backward()
        optimizer_1.update()

    elapsed_time = time() - start
    print("epoch: {}, mean loss: {}, elapsed_time: {}".format(epoch,
                                                              sum_loss/num_iter_per_epoch,
                                                              elapsed_time))
    
    if epoch % 20 == 0:
        model.to_cpu()
        chainer.serializers.save_npz("rnn-%03d.npz" % epoch, model)
        model.to_gpu()

## 学習（教師強制を用いない学習）

こちらでは，教師強制を用いない学習を行います．
この方法は，ネットワークから出力された結果（予測された英単語）を次の入力として順番に使用し学習する方法です．


まず，先ほどとは異なるネットワークとして`model_2`を作成し，最適化手法を設定します．
この時のパラメータは同じものを使用します．

最初の入力のみ`x`すなわち正しい単語を入力し，それ以外の入力には前の時刻の結果`y`から求められた単語を入力します．

In [ ]:
# ネットワークの作成
model_2 = RNNLM(n_vocab=num_vocab, n_units=num_units)
model_2.to_gpu()

# 最適化手法の設定
optimizer_2 = chainer.optimizers.MomentumSGD(lr=1.0, momentum=0.9)
optimizer_2.setup(model_2)
optimizer_2.add_hook(GradientClipping(5.0))


# ミニバッチサイズ・エポック数．学習データ数の設定
batch_size = 128
epoch_num = 100
train_data_num = train_x.shape[0]
num_iter_per_epoch = int(train_data_num / batch_size)


# 学習の実行
start = time()
for epoch in range(1, epoch_num + 1):
    
    sum_loss = 0
    
    perm = xp.random.permutation(train_data_num)
    
    for i in range(0, train_data_num, batch_size):
        
        x = Variable(cuda.to_gpu(train_x[perm[i:i+batch_size]]))
        t = Variable(cuda.to_gpu(train_y[perm[i:i+batch_size]]))

        pred = None
        accum_loss = 0
        model_2.reset_state()

        for idx_window in range(bproplen):
            
            if idx_window == 0:
                y = model_2(x[:, idx_window])
            else:
                y = model_2(pred)
                    
            pred = F.argmax(y, axis=1)
            
            loss = F.softmax_cross_entropy(y, t[:, idx_window])
            accum_loss += loss
            sum_loss += loss.data
            
        optimizer_2.target.cleargrads()
        accum_loss.backward()
        accum_loss.unchain_backward()
        optimizer_2.update()
            
    elapsed_time = time() - start
    print("epoch: {}, mean loss: {}, elapsed_time: {}".format(epoch,
                                                              sum_loss/num_iter_per_epoch,
                                                              elapsed_time))

## テスト
学習したネットワークモデルを用いて評価を行います．


### 1. 教師強制ありのモデル


In [ ]:
true_sentense = []
pred_sentense = []

with chainer.using_config('train', False), chainer.using_config('enable_backprop', False):
    pred_word = None
    for i in range(10):
        x = Variable(cuda.to_gpu(test[i].reshape(1, 1)))
        y = model_1(x)
        pred = F.argmax(y, axis=1)
        
        true_word = inverse_vocab[int(cuda.to_cpu(x.data[0]))]
        pred_word = inverse_vocab[int(cuda.to_cpu(pred.data[0]))]
        true_sentense.append(true_word)
        pred_sentense.append(pred_word)
    
    for i in range(10):
        x = Variable(cuda.to_gpu(test[i].reshape(1, 1)))
        y = model_1(pred)
        pred = F.argmax(y, axis=1)
        
        true_word = inverse_vocab[int(cuda.to_cpu(x.data[0]))]
        pred_word = inverse_vocab[int(cuda.to_cpu(pred.data[0]))]
        true_sentense.append(true_word)
        pred_sentense.append(pred_word)

print(' '.join(true_sentense[0:10]))
print(' '.join(pred_sentense[9:]))

### 2. 教師強制なしのモデル

In [ ]:
true_sentense = []
pred_sentense = []

with chainer.using_config('train', False), chainer.using_config('enable_backprop', False):
    pred_word = None
    for i in range(10):
        x = Variable(cuda.to_gpu(test[i].reshape(1, 1)))
        y = model_2(x)
        pred = F.argmax(y, axis=1)
        
        true_word = inverse_vocab[int(cuda.to_cpu(x.data[0]))]
        pred_word = inverse_vocab[int(cuda.to_cpu(pred.data[0]))]
        true_sentense.append(true_word)
        pred_sentense.append(pred_word)
    
    for i in range(10):
        x = Variable(cuda.to_gpu(test[i].reshape(1, 1)))
        y = model_2(pred)
        pred = F.argmax(y, axis=1)
        
        true_word = inverse_vocab[int(cuda.to_cpu(x.data[0]))]
        pred_word = inverse_vocab[int(cuda.to_cpu(pred.data[0]))]
        true_sentense.append(true_word)
        pred_sentense.append(pred_word)

print(' '.join(true_sentense[0:10]))
print(' '.join(pred_sentense[9:]))

## 課題
1. ネットワークのLSTM層の数を変更した際の性能変化を確認しましょう
2. ネットワークのLSTMをGRUに変更して性能の変化を確認しましょう